In [1]:
import os
import sys
import io
import datetime
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tnrange

import tensorflow as tf

from sklearn import model_selection

sys.path.append('%s/lib' % (os.path.abspath('..')))
from SyntheticLCGenerator import synthetic_light_curve_generator

In [2]:
main_path = os.path.abspath('..')

## Settings

In [10]:
# data
num_samples = 214000
seq_length = 50
num_signals = 1
use_time = False
use_err = False
n_feat = 3
if (use_time and not use_err) or (not use_time and use_err):
    n_feat = 2
elif not use_time and not use_err:
    n_feat = 1
data_name = 'SynSine_time%s_err%s' % (str(use_time)[0], str(use_err)[0])

# GAN architecture
model_name = 'RGAN'
learning_rate = 0.1
latent_dim = 5
D_hidden_units = 100
G_hidden_units = 100
D_output = 1
dropout = 0.1
batch_size = 28
n_epochs = 200
N_gen_feat = n_feat
save_interval = 5
viz_interval = 1
batch_count = round(7000*.6 / batch_size)
n_examples = 8
gen_lc_len = n_obs

## Load Data

In [4]:
def load_synthetic_time_series(data_path=None, n_samples=14000, seq_length=50,
                               time_span=4, n_bands=1, n_signals=1, SNR_min=5,
                               f0_interval='narrow',
                               use_time=True, use_err=True):

    if f0_interval == 'narrow':
        f0_inter=[1/10., 1/1.]
    elif f0_interval == 'wide':
        f0_inter=[1/100., 1/0.01]
    else:
        print('Plese define frequency interval...')
        return
    if os.path.exists(data_path):
        print('Loading from: ', data_path)
        aux = np.load(data_path).item()
        samples = aux['samples']
        labels = np.array(aux['labels'])[:, None]
        periods = aux['periods']
        if n_bands == 1:
            samples = samples.reshape(samples.shape[0], samples.shape[2],
                                      samples.shape[3])
        del aux
        if use_time and not use_err:
            samples = samples[:, :, 0:2]
        if not use_time and not use_err:
            samples = samples[:, :, 1:2]
    else:
        print('File does not exist...')
    return samples, labels, periods


def load_real_time_series(data_path=None, survey='EROS2', n_bands=2,
                          use_time=True, use_err=True):
    if os.path.exists(data_path):
        print('Loading from: ', data_path)
        aux = joblib.load(data_path)
        lcs = aux['lcs']
        meta = aux['meta']
        del aux
        lcs = np.stack([x.values for x in lcs])
        print(lcs.shape)
        if n_bands == 1:
            lcs = lcs[:, :, 0:3]
        if use_time and not use_err:
            lcs = lcs[:, :, 0:2]
        if not use_time and not use_err:
            lcs = lcs[:, :, 1:2]
    else:
        print('No file...')
        return
    return lcs, meta


def normalize(data, norm_time=False, scale_to=[0, 1], n_feat=2):
    normed = np.zeros_like(data)
    for i, lc in enumerate(data):     
        for f in range(n_feat):
            normed[i, :, f] = lc[:, f]
            ## normalize time if asked
            if f == 0 and norm_time:
                normed[i, :, f] = (lc[:, f] - np.min(lc[:, f])) / \
                                  (np.max(lc[:, f]) - np.min(lc[:, f]))
            ## normalize other feature values
            if f > 0:
                normed[i, :, f] = (lc[:, f] - np.min(lc[:, f])) / \
                                  (np.max(lc[:, f]) - np.min(lc[:, f]))
            ## scale feature values if asked
            if scale_to != [0, 1]:
                normed[i, :, f] = (normed[i, :, f] * (scale_to[1] - scale_to[0])) + scale_to[0]
    return normed


def standarize(data, stand_time=False):
    standar = np.zeros_like(data)
    for i, lc in enumerate(data):
        standar[i, :, 1] = (lc[:, 1] - np.mean(lc[:, 1])) / np.std(lc[:, 1])
        if stand_time:
            standar[i, :, 0] = (lc[:, 0] - np.mean(lc[:, 0])) / np.std(lc[:, 0])
        else:
            standar[i, :, 0] = lc[:, 0]
    return standar

In [11]:
data_path = ('%s/data/synthetic/sine_nsamples%i_seqlength%i_nbands%i_nsig%i_timespan%i_SNR%i_f0%s.npy'
             % (main_path, num_samples*2, seq_length, num_signals, 1, 4, 40, 'narrow'))
lightcurves, labels, periods = load_synthetic_time_series(data_path, 
                                                      use_time=use_time,
                                                      use_err=use_err)
lightcurves = normalize(lightcurves[:14000], norm_time=True, scale_to=[-1, 1], 
                    n_feat=num_generated_features)
labels = labels[:14000]
periods = periods[:14000]
print('Samples shape: ', lightcurves.shape)

train, test, p_train, p_test = model_selection.train_test_split(lightcurves, labels,
                                                                train_size=0.6)
test, vali, p_test, p_vali = model_selection.train_test_split(test, p_test,
                                                              train_size=0.5)
lightcurves = {}
lightcurves['train'] = train
lightcurves['vali'] = vali
lightcurves['test'] = test
labels = {}
labels['train'] = p_train
labels['vali'] = p_vali
labels['test'] = p_test

del train, test, vali

File does not exist...


UnboundLocalError: local variable 'samples' referenced before assignment

## Models 

##### Ops

In [6]:
def get_noise(n_samples, n_timesteps=50, latent_dim=5, use_time=True):
    latent = np.random.normal(size=[n_samples, n_timesteps, latent_dim])
    if use_time:
        latent[:, :, 0] = np.linspace(0, 1.0, num=n_timesteps)
    return latent

#### Generator

In [58]:
def generator(z, reuse=False):
    with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
            
        with tf.variable_scope("lstm1"):
            cell = tf.contrib.rnn.LSTMCell(num_units=G_hidden_units,
                                           state_is_tuple=True, 
                                           initializer=None,
                                           reuse=reuse)
            rnn_output, rnn_states = tf.nn.dynamic_rnn(cell=cell, 
                                                       dtype=tf.float32,
                                                       sequence_length=[n_obs]*batch_size,
                                                       inputs=z)
            print(rnn_output.shape)
        with tf.variable_scope("reshape"):
            rnn_output_2d = tf.reshape(rnn_output, [-1, G_hidden_units])
            print(rnn_output_2d.shape)

        with tf.variable_scope("dense"):
            logits_2d = tf.layers.Dense(rnn_output_2d, 2, 
                                        activation_fn=tf.truncated_normal_initializer())
            print(logits_2d.shape)
            output_2d = tf.nn.tanh(logits_2d)

        with tf.variable_scope("reshape"):
            out = tf.reshape(output_2d, [-1, n_obs, n_feat])
    return out

#### Discriminator

In [59]:
def discriminator(x, reuse=False):
    with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):

        with tf.variable_scope("lstm1"):
            cell = tf.contrib.rnn.LSTMCell(num_units=D_hidden_units,
                                           initializer=None,
                                           state_is_tuple=True,
                                           reuse=reuse)
            rnn_output, rnn_states = tf.nn.dynamic_rnn(cell=cell, 
                                                       dtype=tf.float32,
                                                       sequence_length=[n_obs]*batch_size,
                                                       inputs=x)

        with tf.variable_scope("dense"):
            logits = tf.contrib.layers.fully_connected(rnn_output, 1, activation_fn=None)

        with tf.variable_scope("calss_prob"):
            
            out = tf.nn.sigmoid(logits)
            
    return out, logits

## Graph Optimization

#### Inputs

In [60]:
## Real Input
X = tf.placeholder(tf.float32, shape=(batch_size, n_obs, n_feat), name='real_lc')
print(X)
## Latent Variables / Noise
Z = tf.placeholder(tf.float32, shape=(batch_size, n_obs, latent_dim), name='noise')
print(Z)

Tensor("real_lc_9:0", shape=(28, 50, 1), dtype=float32)
Tensor("noise_9:0", shape=(28, 50, 5), dtype=float32)


#### Outputs

In [61]:
# Generator
G_sample = generator(Z)
print('G_sample', G_sample)
# Discriminator
D_real = discriminator(X)
D_fake = discriminator(G_sample)
print(D_real)
print(D_fake)

(28, 50, 100)


ValueError: Trying to share variable generator/dense/fully_connected/weights, but specified shape (100, 2) and found shape (100, 100).

#### Losses

In [33]:
tf.ones((50, 1)), tf.ones_like(D_real)

(<tf.Tensor 'ones_3:0' shape=(50, 1) dtype=float32>,
 <tf.Tensor 'ones_like_3:0' shape=(2, ?, 50, 1) dtype=float32>)

In [11]:
# Generator
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=D_fake, labels=tf.ones_like(D_fake)))

# Discriminator
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=D_real, labels=tf.ones_like(D_real)))

D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=D_fake, labels=tf.zeros_like(D_fake)))

D_loss = D_loss_real + D_loss_fake

#### Optimizers

In [12]:
# Obtain trainable variables for both networks
train_vars = tf.trainable_variables()

G_vars = [var for var in train_vars if 'generator' in var.name]
D_vars = [var for var in train_vars if 'discriminator' in var.name]

num_epochs = 200

In [13]:
G_opt = tf.train.AdamOptimizer(
    learning_rate=learning_rate).minimize(G_loss, var_list=G_vars,)
D_opt = tf.train.AdamOptimizer(
    learning_rate=learning_rate).minimize(D_loss, var_list=D_vars,)

## Helping functions

In [14]:
# Create a wall of generated time series
def plot_generated_time_series(epoch, generated_lc, 
                               test_lc=None, examples=8,
                               dim=(2, 4), figsize=(16, 4),
                               use_time=True, use_err=True):

    fig, ax = plt.subplots(nrows=dim[0], ncols=dim[1], figsize=figsize)
    fig.suptitle('Real (black), Gen (blue) LCs Epoch %i' % (epoch), 
                 fontsize=20, y=1.05)
    for i in range(dim[0]):
        for j in range(dim[1]):
            if use_time and use_err:
                if j == 0 and test_lc is not None:
                    ax[i,j].errorbar(test_lc[i, :, 0], test_lc[i, :, 1],
                                     yerr=test_lc[i, :, 2], fmt='k.')
                else:
                    ax[i,j].errorbar(generated_lc[(j+i) + j, :, 0], 
                                     generated_lc[(j+i) + j, :, 1],
                                     yerr=generated_lc[(j+i) + j, :, 2], 
                                     fmt='.', c='royalblue')
            elif use_time and not use_err:
                if j == 0 and test_lc is not None:
                    ax[i,j].plot(test_lc[i, :, 0], test_lc[i, :, 1], 'k.')
                else:
                    ax[i,j].plot(generated_lc[(j+i) + j, :, 0], 
                                 generated_lc[(j+i) + j, :, 1], '.', 
                                 c='royalblue')
            elif not use_time and not use_err:
                if j == 0 and test_lc is not None:
                    ax[i,j].plot(test_lc[i, :], 'k.')
                else:
                    ax[i,j].plot(generated_lc[(j+i) + j, :], '.', 
                                 c='royalblue')
    plt.tight_layout()
    # plt.savefig('gan_generated_image_epoch_%d.png' % epoch)
    # buf = io.BytesIO()
    # plt.savefig(buf, format='png')
    # buf.seek(0)
    plt.show()
    return

def write_log_scalar(callback, names, logs, step):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, step)
        callback.writer.flush()
        
def write_log_plot(callback, buf, step):
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    summary = tf.summary.image("Generated_Image", image,
                               max_outputs=1)
    callback.writer.add_summary(tf.keras.backend.eval(summary), step)
    callback.writer.flush()

## Training

#### Testing

In [15]:
test_noise = get_noise(n_examples, n_timesteps=n_obs, 
                       latent_dim=latent_dim, use_time=use_time)

#### Train

In [19]:
# Start interactive session
session = tf.InteractiveSession()
# Init Variables
tf.global_variables_initializer().run()

# Iterate through epochs
for epoch in range(n_epochs):
    print('-------- Epoch %i/%i --------' % (epoch, n_epochs))
    for n_batch in tnrange(batch_count, desc='# batch'):

        # 1. Train Discriminator
        X_batch = x_train[n_batch * batch_size: (n_batch + 1) * batch_size]
        Z_batch = get_noise(n_examples, n_timesteps=n_obs,
                            latent_dim=latent_dim, use_time=use_time)
        feed_dict = {X: X_batch, Z: Z_batch}
        _, d_error, d_pred_real, d_pred_fake = session.run([D_opt, D_loss,
                                                            D_real, D_fake],
                                                           feed_dict=feed_dict)

        # 2. Train Generator
        Z_batch_g = get_noise(n_examples, n_timesteps=n_obs,
                              latent_dim=latent_dim, use_time=use_time)
        feed_dict = {Z: Z_batch_g}
        _, g_error = session.run([G_opt, G_loss],
                                 feed_dict=feed_dict)

        if n_batch % 100 == 0:
            display.clear_output(True)
            # Generate images from test noise
            test_lc = session.run(G_sample, feed_dict={Z: test_noise})
            _ = plot_generated_time_series(e, test_lc,
                                           test_lc=x_test[np.random.randint(0,len(x_test),2)])
session.InteractiveSession.close()

-------- Epoch 0/200 --------


/Users/jorgetil/miniconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


InvalidArgumentError: assertion failed: [Expected shape for Tensor generator/lstm1/rnn/sequence_length:0 is ] [8] [ but saw shape: ] [28]
	 [[node generator/lstm1/rnn/Assert/Assert (defined at <ipython-input-7-43be46e4a520>:12)  = Assert[T=[DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](generator/lstm1/rnn/All, generator/lstm1/rnn/Assert/Assert/data_0, generator/lstm1/rnn/stack, generator/lstm1/rnn/Assert/Assert/data_2, generator/lstm1/rnn/Shape_1)]]

Caused by op 'generator/lstm1/rnn/Assert/Assert', defined at:
  File "/Users/jorgetil/miniconda3/envs/keras/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jorgetil/miniconda3/envs/keras/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jorgetil/miniconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jo